# Lending club loan prediction

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

pd.options.display.max_columns = 100

In [2]:
data = pd.read_csv('loans_2007.csv')
print(data.shape)
data.head()

(42538, 52)


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


# Data cleaning

Dropping columns that are randomly generated or leak info from future 

In [3]:
drop_cols = ['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title','issue_d']
data = data.drop(drop_cols, axis=1)
data.shape

(42538, 44)

In [4]:
drop_cols = ['zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp']
data = data.drop(drop_cols, axis=1)
assert(data.shape[1] == 38)

In [5]:
drop_cols = ['total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt']
data = data.drop(drop_cols, axis=1)
assert(data.shape[1] == 32)

### Picking target column: loan_status

In [6]:
data.loan_status.value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

### Explanation of loan_status values:

* Fully Paid	33136	Loan has been fully paid off.
* Charged Off	5634	Loan for which there is no longer a reasonable expectation of further payments.
* Does not meet the credit policy. Status:Fully Paid	1988	While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
* Does not meet the credit policy. Status:Charged Off	761	While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
* In Grace Period	20	The loan is past due but still in the grace period of 15 days.
* Late (16-30 days)	8	Loan hasn't been paid in 16 to 30 days (late on the current payment).
* Late (31-120 days)	24	Loan hasn't been paid in 31 to 120 days (late on the current payment).
* Current	961	Loan is up to date on current payments.
* Default	3	Loan is defaulted on and no payment has been made for more than 121 days.

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. You can read about the difference here.  Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one.

In [7]:
replace_dict={"loan_status": {"Fully Paid": 1, "Charged Off":0}}
data = data.replace(replace_dict)

In [8]:
temp_data = data[(data['loan_status']== 1) | (data['loan_status'] == 0)]
temp_data.loan_status.value_counts()

1    33136
0     5634
Name: loan_status, dtype: int64

In [9]:
data = temp_data
data['loan_status'] = data['loan_status'].astype(int)

Drop all columns that only have one value in them 

In [10]:
drop_cols = []

for each in data.columns:
    series = data[each]
    series = series.dropna()
    if len(series.unique()) == 1:
        drop_cols.append(each)
print(drop_cols)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [11]:
old_col_number = data.shape[1]

data = data.drop(drop_cols,axis = 1)
assert(data.shape[1] == old_col_number - len(drop_cols))

### Filtering null columns

In [12]:
data.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

We will assume 'na' for emp_length implies working at current job < 1 year, and we will assume 'na' for pub_rec_bankruptcies implies 0 bankrupcies

In [13]:
data = data.fillna({"emp_length": 0, "pub_rec_bankruptcies": 0})
data.isnull().sum()

loan_amnt                0
term                     0
int_rate                 0
installment              0
emp_length               0
home_ownership           0
annual_inc               0
verification_status      0
loan_status              0
purpose                  0
title                   11
addr_state               0
dti                      0
delinq_2yrs              0
earliest_cr_line         0
inq_last_6mths           0
open_acc                 0
pub_rec                  0
revol_bal                0
revol_util              50
total_acc                0
last_credit_pull_d       2
pub_rec_bankruptcies     0
dtype: int64

Dropping remaining rows with 'na' values

In [14]:
print(data.shape)
data = data.dropna(axis=0)
print(data.shape)

(38770, 23)
(38707, 23)


In [15]:
data.isnull().sum()

loan_amnt               0
term                    0
int_rate                0
installment             0
emp_length              0
home_ownership          0
annual_inc              0
verification_status     0
loan_status             0
purpose                 0
title                   0
addr_state              0
dti                     0
delinq_2yrs             0
earliest_cr_line        0
inq_last_6mths          0
open_acc                0
pub_rec                 0
revol_bal               0
revol_util              0
total_acc               0
last_credit_pull_d      0
pub_rec_bankruptcies    0
dtype: int64

### Converting to numerical data

In [16]:
data.dtypes.value_counts()

object     11
float64    11
int64       1
dtype: int64

In [17]:
text_data = data.select_dtypes(include=['object'])
text_data.dtypes.value_counts()

object    11
dtype: int64

In [18]:
text_data.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-1996,21%,Apr-2016
5,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-2004,28.3%,Jan-2016


In [19]:
for each in text_data.columns:
    print(each)
    print(text_data[each].value_counts())
    print()

term
 36 months    29040
 60 months     9667
Name: term, dtype: int64

int_rate
 10.99%    928
 11.49%    793
  7.51%    787
 13.49%    760
  7.88%    725
  7.49%    652
  9.99%    593
  7.90%    574
  5.42%    573
 11.71%    563
 11.99%    492
 10.37%    469
 12.69%    452
  6.03%    447
  8.49%    440
 12.99%    417
  5.79%    410
 12.42%    407
 10.65%    404
  7.29%    397
  6.62%    396
 11.86%    391
  8.90%    386
  9.63%    378
 10.59%    362
  9.91%    357
  5.99%    347
 14.27%    342
  7.14%    340
  6.99%    336
          ... 
 21.82%      2
 17.90%      2
 13.84%      2
 20.20%      2
 15.01%      2
 15.83%      2
 17.15%      2
 14.70%      2
 14.88%      2
 17.03%      2
 15.38%      2
 24.40%      1
 14.67%      1
 21.48%      1
 16.20%      1
 16.15%      1
 17.54%      1
 22.64%      1
 17.44%      1
 10.64%      1
 17.34%      1
 18.36%      1
 20.52%      1
 16.01%      1
 18.72%      1
 16.33%      1
 24.59%      1
 16.71%      1
 16.96%      1
 17.46%      1
Name:

Name: revol_util, Length: 1088, dtype: int64

last_credit_pull_d
Jun-2016    10727
Mar-2016     1057
Apr-2016      864
Feb-2013      838
May-2016      735
Feb-2016      704
Jan-2016      633
Dec-2015      626
Mar-2013      574
Dec-2014      556
Mar-2014      555
Jul-2014      546
Sep-2014      540
Jul-2015      522
Feb-2014      521
Oct-2014      515
Aug-2014      515
Jan-2014      512
Sep-2015      505
Nov-2015      500
Nov-2014      492
Apr-2014      482
Oct-2015      478
Nov-2013      470
May-2014      466
Jun-2014      456
May-2013      447
Jan-2015      427
Apr-2013      422
Dec-2013      418
            ...  
Jan-2013       34
Oct-2009       33
Nov-2009       28
Jan-2010       27
Aug-2009       19
Feb-2009       16
Aug-2007       15
Apr-2009       15
Jan-2009       14
Jul-2009       13
Jun-2009       13
Jun-2007       12
Mar-2009       10
May-2009       10
Sep-2009       10
Aug-2008        9
Oct-2008        9
Dec-2008        5
Sep-2008        5
Mar-2008        4
Feb-2008        3

In [20]:
data = data.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'], axis=1)
data

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,pub_rec_bankruptcies
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.00,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7%,9.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.00,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4%,4.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.00,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5%,10.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.00,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21%,37.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.00,Source Verified,1,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.3%,12.0,0.0
6,7000.0,60 months,15.96%,170.08,8 years,RENT,47004.00,Not Verified,1,debt_consolidation,23.51,0.0,1.0,7.0,0.0,17726.0,85.6%,11.0,0.0
7,3000.0,36 months,18.64%,109.43,9 years,RENT,48000.00,Source Verified,1,car,5.35,0.0,2.0,4.0,0.0,8221.0,87.5%,4.0,0.0
8,5600.0,60 months,21.28%,152.39,4 years,OWN,40000.00,Source Verified,0,small_business,5.55,0.0,2.0,11.0,0.0,5210.0,32.6%,13.0,0.0
9,5375.0,60 months,12.69%,121.45,< 1 year,RENT,15000.00,Verified,0,other,18.08,0.0,0.0,2.0,0.0,9279.0,36.5%,3.0,0.0
10,6500.0,60 months,14.65%,153.45,5 years,OWN,72000.00,Not Verified,1,debt_consolidation,16.12,0.0,2.0,14.0,0.0,4032.0,20.6%,23.0,0.0


In [21]:
# convert percentages to numeric data
data['int_rate'] = data['int_rate'].apply(lambda x: x.rstrip('%')).astype(float)
data['revol_util'] = data['revol_util'].apply(lambda x: x.rstrip('%')).astype(float)

In [22]:
# converting emp_length to numeric

mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

data = data.replace(mapping_dict)
data.emp_length.value_counts()

10    8545
0     5545
2     4303
3     4022
4     3353
5     3202
1     3176
6     2177
7     1714
8     1442
9     1228
Name: emp_length, dtype: int64

In [23]:
data

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,pub_rec_bankruptcies
0,5000.0,36 months,10.65,162.87,10,RENT,24000.00,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.70,9.0,0.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.00,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.40,4.0,0.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.00,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.50,10.0,0.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.00,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.00,37.0,0.0
5,5000.0,36 months,7.90,156.46,3,RENT,36000.00,Source Verified,1,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.30,12.0,0.0
6,7000.0,60 months,15.96,170.08,8,RENT,47004.00,Not Verified,1,debt_consolidation,23.51,0.0,1.0,7.0,0.0,17726.0,85.60,11.0,0.0
7,3000.0,36 months,18.64,109.43,9,RENT,48000.00,Source Verified,1,car,5.35,0.0,2.0,4.0,0.0,8221.0,87.50,4.0,0.0
8,5600.0,60 months,21.28,152.39,4,OWN,40000.00,Source Verified,0,small_business,5.55,0.0,2.0,11.0,0.0,5210.0,32.60,13.0,0.0
9,5375.0,60 months,12.69,121.45,0,RENT,15000.00,Verified,0,other,18.08,0.0,0.0,2.0,0.0,9279.0,36.50,3.0,0.0
10,6500.0,60 months,14.65,153.45,5,OWN,72000.00,Not Verified,1,debt_consolidation,16.12,0.0,2.0,14.0,0.0,4032.0,20.60,23.0,0.0


In [24]:
nominal_cols = ['home_ownership', 'verification_status', 'purpose', 'term']
dummy_df = pd.DataFrame()

for each in nominal_cols:
    dummies = pd.get_dummies(data[each],prefix=each)
    data = pd.concat([data, dummies], axis=1)
    data = data.drop(each, axis=1)
    
data.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,pub_rec_bankruptcies,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


# Modeling

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38707 entries, 0 to 39785
Data columns (total 39 columns):
loan_amnt                              38707 non-null float64
int_rate                               38707 non-null float64
installment                            38707 non-null float64
emp_length                             38707 non-null int64
annual_inc                             38707 non-null float64
loan_status                            38707 non-null int64
dti                                    38707 non-null float64
delinq_2yrs                            38707 non-null float64
inq_last_6mths                         38707 non-null float64
open_acc                               38707 non-null float64
pub_rec                                38707 non-null float64
revol_bal                              38707 non-null float64
revol_util                             38707 non-null float64
total_acc                              38707 non-null float64
pub_rec_bankruptcies       

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import *

In [27]:
features = data.drop(['loan_status'], axis=1)
target = data['loan_status']

In [28]:
def model(clf, cv = 3):
    predict = cross_val_predict(clf, features, target, cv = cv)
    predict = pd.Series(predict)
    tp = len(predict[(predict == 1) & (target == 1)])
    fp = len(predict[(predict == 1) & (target == 0)])
    fn = len(predict[(predict == 0) & (target == 1)])
    tn = len(predict[(predict == 0) & (target == 0)])
        
    tpr = tp / (tp + fn)
    fpr = fp / (tn + fp)
    return tpr, fpr

In [29]:
print(model(LogisticRegression(class_weight=None)))

(0.9983538327742577, 0.9974250505793636)


Without using balanced class weights, our model achieves good accuracy by just predicting '1' everytime (since this is so much more common than '0')

In [30]:
print(model(LogisticRegression(class_weight='balanced')))

(0.622934526028078, 0.6270001839249586)


We significantly improve the FPR by using balanced class weights

In [31]:
print(model(LogisticRegression(class_weight={0: 10, 1: 1})))

(0.22061746800844825, 0.2286187235607872)


We dropped false positive rate even further by imposing manual 10x penalty using class_weight parameter.  This is good as it helps us avoid picking loans that will eventually default, but we also miss 80% of loans that will be paid off!

In [32]:
print(model(RandomForestClassifier(n_estimators=10, random_state = 1, class_weight=None)))
print(model(RandomForestClassifier(n_estimators=10, random_state = 1, class_weight='balanced')))
print(model(RandomForestClassifier(n_estimators=10, random_state = 1, class_weight={0:10, 1:1})))

(0.945428003478693, 0.9407761633253633)
(0.9626661697105231, 0.9630310833180062)
(0.9673872530749161, 0.9628471583593894)


Hmm, even balanced RF classifier and penalized RF has very high FPR

In [33]:
print(model(RandomForestClassifier(n_estimators=100, random_state = 1, class_weight=None)))
print(model(RandomForestClassifier(n_estimators=100, random_state = 1, class_weight='balanced')))
print(model(RandomForestClassifier(n_estimators=100, random_state = 1, class_weight={0:10, 1:1})))

(0.9915206857994782, 0.9908037520691558)
(0.9965213069946577, 0.9972411256207467)
(0.9968629643433967, 0.9963215008276624)


In [34]:
print(model(MLPClassifier(hidden_layer_sizes=(25,25),max_iter=1000,random_state=1)))
print(model(MLPClassifier(hidden_layer_sizes=(50,50),max_iter=1000,random_state=1)))
print(model(MLPClassifier(hidden_layer_sizes=(100,100),max_iter=1000,random_state=1)))

(0.9982295937383526, 0.9970572006621299)
(0.9153000372717107, 0.9166819937465515)
(0.995527394707417, 0.9963215008276624)


Achieving balance by adding in extra '0' rows

In [38]:
copied_data = data.copy()
ones = copied_data[copied_data['loan_status'] == 1]
zeros = copied_data[copied_data['loan_status'] == 0]
extra_zeros = ones.copy()
for each in range(5):
    extra_zeros = pd.concat([extra_zeros, zeros])
    
extra_zeros = extra_zeros.reset_index(drop=True)
extra_zeros.shape

(61167, 39)

In [39]:
features = extra_zeros.drop(['loan_status'], axis=1)
target = extra_zeros['loan_status']

In [41]:
print(model(LogisticRegression()))
print(model(RandomForestClassifier(n_estimators=10, random_state = 1)))
print(model(RandomForestClassifier(n_estimators=100, random_state = 1)))
print(model(MLPClassifier(hidden_layer_sizes=(25,25),max_iter=1000,random_state=1)))
print(model(MLPClassifier(hidden_layer_sizes=(50,50),max_iter=1000,random_state=1)))
print(model(MLPClassifier(hidden_layer_sizes=(100,100),max_iter=1000,random_state=1)))

(0.6349873081107216, 0.43882457702582367)
(0.7410854587211411, 0.0)
(0.8318626858455216, 0.0)
(0.9271425117853258, 0.85506678539626)
(0.4711712800676901, 0.386179875333927)
(0.3289616825818929, 0.2602315227070347)


Achieving balance by deleting '1' rows

In [44]:
copied_data = data.copy()
ones = copied_data[copied_data['loan_status'] == 1]
zeros = copied_data[copied_data['loan_status'] == 0]
ones = ones.sample(frac=1)
num_zeros = zeros.shape[0]
ones = ones.iloc[:num_zeros]
data = pd.concat([ones, zeros])
data = data.sample(frac=1)
print(data.shape)

(11230, 39)


In [45]:
features = data.drop(['loan_status'], axis=1)
target = data['loan_status']

In [46]:
print(model(LogisticRegression()))
print(model(RandomForestClassifier(n_estimators=10, random_state = 1)))
print(model(RandomForestClassifier(n_estimators=100, random_state = 1)))
print(model(MLPClassifier(hidden_layer_sizes=(25,25),max_iter=1000,random_state=1)))
print(model(MLPClassifier(hidden_layer_sizes=(50,50),max_iter=1000,random_state=1)))
print(model(MLPClassifier(hidden_layer_sizes=(100,100),max_iter=1000,random_state=1)))

(0.4638888888888889, 0.4658959537572254)
(0.4013888888888889, 0.4346820809248555)
(0.4826388888888889, 0.48728323699421966)
(0.35625, 0.32543352601156067)
(0.33611111111111114, 0.37052023121387284)
(0.6375, 0.6479768786127168)
